# Extracting Data From the WMO app

## 1 Insturactions of the WMO

Use the following steps to carry out the project
- Use this link (https://community.wmo.int/en/members/profiles to extract data on
heatwaves warning system/services, see example below, if heatwave is in the list of
warnings provided, then indicate a yes in the provided excel sheet (for the corresponding
country in column B in the excel sheet called “Heatwaves Warning Systems (WMO data)”).


full screenmode : https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9


## 2  Required Libraries

In [87]:
#The webdriver module in Selenium allows for automated control of web browsers.
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


## 3 Opening up the web browser and printing a sample text

The function below opens up chrome browser and waits for 60 minutes to load the data 

input: The website link

Output : the driver and wabdriverwait instance

In [88]:

def openChrome(link):
    # Options customize Chrome WebDriver behavior
    options = Options()

    # Initialize the WebDriver (e.g., Chrome)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Navigate to the desired URL
    driver.get(link)

    # wait for page to load
    wait = WebDriverWait(driver, 60)

    return driver, wait

In [89]:
link = "https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9"
driver_instance, wait  = openChrome(link)

Using the driver instance that we created print an sample text

In [90]:
# Locate all div elements with class "textbox"
div_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="textbox"]')))

# Iterate over the list of elements and print the text content of each
for div_element in div_elements:
    print(div_element.text)
    print("-----")  # Just a separator for clarity

# driver_instance.quit()

Member Profiles
-----
The information contained in these profiles have been provided by WMO Members. The information is based on a self-assessment exercise. The date displayed in each section corresponds with the date in which the information was provided to WMO. 

If you find any issue or inconsistency with the information displayed, please contact community@wmo.int.
-----
Permanent Representative:
-----
Hydrological Adviser:
-----
National Meteorological Service:
-----
Responsible Ministry:
-----


## 4. Clicking on the Early Warning Services 

In [91]:
#EC stands for "Expected Conditions." It is a set of predefined conditions that the Selenium WebDriver uses.
#presence_of_element_located is one of many expected conditions (EC). It checks if an element is present on the DOM of a page.
def wait_and_click(driver, wait, css_selector):
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    driver.execute_script("arguments[0].click();", element)

In [92]:
css_selector = 'a[title=" Early Warning Services"]'
wait_and_click(driver_instance, wait, css_selector)

## 5 - Get Warnings List

In [93]:
def saveWarnings(driver, css_selector):
    #'.title' is the class name of the element that contains the warning text
    warnings= driver_instance.find_elements(By.CSS_SELECTOR, css_selector)
    return warnings

warnings = saveWarnings(driver_instance, '.title')


In [94]:

for warning in warnings:
    print(warning.text)

## 6 getting all country names 

In [95]:

def getCountryNames(driver, wait):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    loaded_countries = set()
    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()
    print(loaded_countries)
    # Close the dropdown menu by clicking the dropdown button again
    dropdown.click()
    return loaded_countries

In [96]:
loaded_countries = getCountryNames(driver_instance, wait)

{'Nepal', 'Hong Kong, China', 'Morocco', 'Kuwait', 'Suriname', 'Cabo Verde', 'Dominican Republic', 'Thailand', 'Russian Federation', 'Singapore', 'Luxembourg', 'Cook Islands', 'New Caledonia', 'Yemen', 'Timor-Leste', 'Mozambique', "Lao People's Democratic Republic", 'Chad', 'Colombia', 'Seychelles', 'Uganda', 'Benin', 'Dominica', 'Nicaragua', 'Canada', 'Guinea-Bissau', 'Ecuador', 'Kenya', 'Madagascar', 'Estonia', 'Togo', 'Cuba', 'Papua New Guinea', 'Switzerland', 'China', 'Fiji', 'Malawi', 'United Republic of Tanzania', 'Malta', 'Mongolia', 'Trinidad and Tobago', 'Zambia', 'United Kingdom of Great Britain and Northern Ireland', 'South Sudan', 'Latvia', 'Somalia', 'Senegal', 'French Polynesia', 'Mexico', 'Cyprus', "Democratic People's Republic of Korea", 'Montenegro', 'Republic of Moldova', 'Georgia', 'Central African Republic', 'Sudan', 'Cameroon', 'Syrian Arab Republic', 'United States of America', 'North Macedonia', 'British Caribbean Territories - British Virgin Islands', 'Barbados'

In [97]:
print(len(loaded_countries ))

198


## 7 Selecting A country

In [99]:
def select_country_from_dropdown(driver, wait, target_country):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    # Clear the search input
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchInput'))
    )
    search_input.clear()

    loaded_countries = set()
    country_selected = False

    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if target country is in the current batch of loaded countries
        if target_country in current_countries:
            for option in current_countries_elements:
                if option.text == target_country:
                    option.click()
                    country_selected = True
                    break

        # If the country has been selected, break out of the loop
        if country_selected:
            break
        
        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()

    return country_selected


In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "United States of America")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "India")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:

warnings = saveWarnings(driver_instance, '.title')

for warning in warnings:
    print(warning.text)

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "Argentina")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:
warnings = saveWarnings(driver_instance, '.title')

for warning in warnings:
    print(warning.text)

## selecting 8 country and getting the information

1- Initiate global variables

    - initiate a dictinary

    - driver, wait

2- Load all the country names

3 -Click on Early warning systems

----- repeat this 8 times ----

4- Click on the desired country name

5- extract warning systems in a list

   (if the list is empty)
   
6- save country name and the list in the map

In [ ]:
# Convert the set to a list
countries_list = list(loaded_countries)

# # Sort the list
# countries_list.sort()

# print(countries_list)
# test= countries_list[0:8]

In [100]:
size = len(countries_list)//20
slice_list = [countries_list[i:i + size] for i in range(0, len(countries_list), size)]

for slice in slice_list:
    print(slice)

['Nepal', 'Hong Kong, China', 'Morocco', 'Kuwait', 'Suriname', 'Cabo Verde', 'Dominican Republic', 'Thailand', 'Russian Federation']
['Singapore', 'Luxembourg', 'Cook Islands', 'New Caledonia', 'Yemen', 'Timor-Leste', 'Mozambique', "Lao People's Democratic Republic", 'Chad']
['Colombia', 'Seychelles', 'Uganda', 'Benin', 'Dominica', 'Nicaragua', 'Canada', 'Guinea-Bissau', 'Ecuador']
['Kenya', 'Madagascar', 'Estonia', 'Togo', 'Cuba', 'Papua New Guinea', 'Switzerland', 'China', 'Fiji']
['Malawi', 'United Republic of Tanzania', 'Malta', 'Mongolia', 'Trinidad and Tobago', 'Zambia', 'United Kingdom of Great Britain and Northern Ireland', 'South Sudan', 'Latvia']
['Somalia', 'Senegal', 'French Polynesia', 'Mexico', 'Cyprus', "Democratic People's Republic of Korea", 'Montenegro', 'Republic of Moldova', 'Georgia']
['Central African Republic', 'Sudan', 'Cameroon', 'Syrian Arab Republic', 'United States of America', 'North Macedonia', 'British Caribbean Territories - British Virgin Islands', 'Bar

In [ ]:
print(slice)

lets divide our country list in to 10 arrays



In [101]:
# test_list = ['Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco']
# print(len(test_list))
my_dict = {}

possible fix 

In [102]:
for country in slice_list[0]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)
        


Nepal
country been selected successfully!
Flood
Lightning
Rain/Wet Spell
Snow
Thunderstorms/Squall lines
Wind
Yes
No
Yes
No
Yes
Yes
No
No
Flood
Lightning
Rain/Wet Spell
Snow
Thunderstorms/Squall lines
Wind
Yes
No
Yes
No
Yes
Yes
No
No
-----
Hong Kong, China
country been selected successfully!
Cold wave
Dust storm/Sandstorm
Flood
Frost
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Landslide/Mudslide & Debris flow
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tornado
Yes
Yes
Yes
Different maximum lead times for different warnings, e.g. about one hour for thunderstorm warning, a couple of hours for rainstorm warning, 12 hours or more for TC warning (strong or gale wind signals), etc
Yes
Yes
Yes
Yes
Yes
Nearly 100,000
Cold wave
Dust storm/Sandstorm
Flood
Frost
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Landslide/Mudslide & Debris flow
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Torna

In [103]:
my_dict

{'Nepal': ['Flood',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'No',
  'No'],
 'Hong Kong, China': ['Cold wave',
  'Dust storm/Sandstorm',
  'Flood',
  'Frost',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Landslide/Mudslide & Debris flow',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Tornado',
  'Yes',
  'Yes',
  'Yes',
  'Different maximum lead times for different warnings, e.g. about one hour for thunderstorm warning, a couple of hours for rainstorm warning, 12 hours or more for TC warning (strong or gale wind signals), etc',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Nearly 100,000'],
 'Morocco': ['Cold wave',
  'Dust storm/Sandstorm',
  'Hail',
  'Heat wave',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'No',
  '3 - jours',


In [104]:
for list in range(len(countries_list)):
    print(list)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197


In [ ]:
test_list = ['Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America']
print(len(test_list))
my_dict = {}

In [ ]:
for country in test_list:
    print(country)
    time.sleep(5)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)



10 country

In [ ]:
test_list = [ 'Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America', 'Cyprus']
print(len(test_list))
my_dict = {}

In [ ]:
for country in test_list:
    print(country)
    time.sleep(12)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)

## 9 adjusting so we can run the full loop

In [ ]:
test_list = ['Solomon Islands', 'Ireland', 'Romania', 'Guyana']
print(len(test_list))
my_dict = {}

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "Benin")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, country)

In [ ]:
for country in test_list:
    print(country)
    success = select_country_from_dropdown(driver_instance, wait, country)
    if success:
        print("country been selected successfully!")
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
    else:
        print("Failed to select the country!")
        
